# 🔍 Controle de Qualidade - Pastagens Brasileiras

Sistema avançado de análise de qualidade para imagens sintéticas de pastagens brasileiras.

**Funcionalidades:**
- ✅ Análise automática de qualidade técnica
- ✅ Avaliação de realismo agronômico
- ✅ Consistência sazonal e bioma
- ✅ Comparação com datasets reais
- ✅ Filtragem automática de baixa qualidade

In [ ]:
# Setup
import sys, os
from pathlib import Path
import warnings; warnings.filterwarnings('ignore')

if '/content/img-sinth' not in sys.path:
    sys.path.append('/content/img-sinth')
if not Path('src').exists():
    os.chdir('/content/img-sinth')

import torch, numpy as np, matplotlib.pyplot as plt
from PIL import Image
import json, time
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.auto import tqdm

from src.dataset.quality_metrics import QualityMetrics
from src.diffusion.image_postprocess import ImagePostProcessor

print("✅ Setup concluído!")

# Adicionar projeto ao path
if '/content/img-sinth' not in sys.path:
    sys.path.append('/content/img-sinth')
    
if not Path('src').exists():
    os.chdir('/content/img-sinth')

In [ ]:
def analyze_dataset_quality(dataset_path, sample_size=50):
    """Analisa qualidade de um dataset completo"""
    
    dataset_dir = Path(dataset_path)
    images_dir = dataset_dir / "images"
    
    if not images_dir.exists():
        print(f"❌ Diretório de imagens não encontrado: {images_dir}")
        return
    
    # Inicializar avaliador
    quality_evaluator = QualityMetrics()
    
    # Listar imagens
    image_files = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))
    
    if not image_files:
        print("❌ Nenhuma imagem encontrada")
        return
    
    # Amostra aleatória
    if len(image_files) > sample_size:
        import random
        image_files = random.sample(image_files, sample_size)
    
    print(f"🔍 Analisando {len(image_files)} imagens...")
    
    # Análise das imagens
    results = []
    
    for img_path in tqdm(image_files, desc="Analisando qualidade"):
        try:
            # Carregar imagem
            image = Image.open(img_path)
            
            # Tentar carregar metadados
            metadata_path = dataset_dir / "metadata" / (img_path.stem + ".json")
            metadata = {}
            if metadata_path.exists():
                with open(metadata_path, 'r') as f:
                    data = json.load(f)
                    pasture_config = data.get('pasture_config', {})
                    metadata = {
                        'biome': pasture_config.get('biome'),
                        'season': pasture_config.get('season'),
                        'quality': pasture_config.get('quality')
                    }
            
            # Avaliar qualidade
            report = quality_evaluator.evaluate_image_quality(image, metadata)
            
            results.append({
                'filename': img_path.name,
                'overall_score': report.overall_score,
                'technical_quality': report.technical_quality,
                'agricultural_realism': report.agricultural_realism,
                'seasonal_consistency': report.seasonal_consistency,
                'metadata': metadata
            })
            
        except Exception as e:
            print(f"Erro ao processar {img_path.name}: {e}")
            continue
    
    # Análise dos resultados
    if not results:
        print("❌ Nenhum resultado obtido")
        return
    
    scores = {
        'overall': [r['overall_score'] for r in results],
        'technical': [r['technical_quality'] for r in results],
        'agricultural': [r['agricultural_realism'] for r in results],
        'seasonal': [r['seasonal_consistency'] for r in results]
    }
    
    print("\n📊 RESULTADOS DA ANÁLISE DE QUALIDADE")
    print("=" * 50)
    
    for metric_name, values in scores.items():
        print(f"\n{metric_name.title()}:")
        print(f"   Média: {np.mean(values):.3f} ± {np.std(values):.3f}")
        print(f"   Min/Max: {np.min(values):.3f} / {np.max(values):.3f}")
        print(f"   Mediana: {np.median(values):.3f}")
    
    # Distribuição de qualidade
    overall_scores = scores['overall']
    excellent = sum(1 for s in overall_scores if s > 0.8)
    good = sum(1 for s in overall_scores if 0.6 <= s <= 0.8)
    moderate = sum(1 for s in overall_scores if 0.4 <= s < 0.6)
    poor = sum(1 for s in overall_scores if s < 0.4)
    
    print(f"\n🎯 Distribuição Geral:")
    print(f"   Excelente (>0.8): {excellent} ({excellent/len(results)*100:.1f}%)")
    print(f"   Bom (0.6-0.8): {good} ({good/len(results)*100:.1f}%)")
    print(f"   Moderado (0.4-0.6): {moderate} ({moderate/len(results)*100:.1f}%)")
    print(f"   Ruim (<0.4): {poor} ({poor/len(results)*100:.1f}%)")
    
    # Análise por bioma/estação
    biome_stats = {}
    season_stats = {}
    
    for result in results:
        metadata = result['metadata']
        score = result['overall_score']
        
        if metadata.get('biome'):
            biome = metadata['biome']
            if biome not in biome_stats:
                biome_stats[biome] = []
            biome_stats[biome].append(score)
        
        if metadata.get('season'):
            season = metadata['season']
            if season not in season_stats:
                season_stats[season] = []
            season_stats[season].append(score)
    
    if biome_stats:
        print(f"\n🌿 Qualidade por Bioma:")
        for biome, scores in biome_stats.items():
            print(f"   {biome}: {np.mean(scores):.3f} (n={len(scores)})")
    
    if season_stats:
        print(f"\n🌦️ Qualidade por Estação:")
        for season, scores in season_stats.items():
            print(f"   {season}: {np.mean(scores):.3f} (n={len(scores)})")
    
    # Visualização
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Histograma geral
    axes[0,0].hist(overall_scores, bins=20, alpha=0.7, color='blue')
    axes[0,0].axvline(np.mean(overall_scores), color='red', linestyle='--', label=f'Média: {np.mean(overall_scores):.3f}')
    axes[0,0].set_xlabel('Score Geral')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].set_title('Distribuição de Qualidade Geral')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Comparação de métricas
    metrics_data = [scores['technical'], scores['agricultural'], scores['seasonal']]
    axes[0,1].boxplot(metrics_data, labels=['Técnica', 'Agronômica', 'Sazonal'])
    axes[0,1].set_ylabel('Score')
    axes[0,1].set_title('Comparação de Métricas')
    axes[0,1].grid(True, alpha=0.3)
    
    # Qualidade por bioma
    if biome_stats:
        biome_means = [np.mean(scores) for scores in biome_stats.values()]
        axes[1,0].bar(biome_stats.keys(), biome_means, alpha=0.7, color='green')
        axes[1,0].set_ylabel('Score Médio')
        axes[1,0].set_title('Qualidade por Bioma')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].grid(True, alpha=0.3)
    
    # Qualidade por estação
    if season_stats:
        season_means = [np.mean(scores) for scores in season_stats.values()]
        axes[1,1].bar(season_stats.keys(), season_means, alpha=0.7, color='orange')
        axes[1,1].set_ylabel('Score Médio')
        axes[1,1].set_title('Qualidade por Estação')
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return results

# Interface
dataset_path_widget = widgets.Text(
    value='/content/generated_dataset',
    description='Dataset Path:'
)

sample_size_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=200,
    description='Amostra:'
)

analyze_quality_btn = widgets.Button(
    description='🔍 Analisar Qualidade',
    button_style='info'
)

quality_output = widgets.Output()

def quality_callback(btn):
    with quality_output:
        clear_output()
        analyze_dataset_quality(dataset_path_widget.value, sample_size_widget.value)

analyze_quality_btn.on_click(quality_callback)

display(widgets.VBox([
    widgets.HTML("<h3>🔍 Análise de Qualidade</h3>"),
    dataset_path_widget,
    sample_size_widget,
    analyze_quality_btn,
    quality_output
]))

## 🎯 Filtragem por Qualidade

In [ ]:
def filter_dataset_by_quality(dataset_path, quality_threshold=0.7, output_path=None):
    """Filtra dataset removendo imagens de baixa qualidade"""
    
    if not output_path:
        output_path = f"{dataset_path}_filtered"
    
    dataset_dir = Path(dataset_path)
    output_dir = Path(output_path)
    
    print(f"🎯 Filtrando dataset com threshold {quality_threshold}...")
    print(f"Origem: {dataset_dir}")
    print(f"Destino: {output_dir}")
    
    # Criar estrutura de saída
    (output_dir / "images").mkdir(parents=True, exist_ok=True)
    (output_dir / "metadata").mkdir(parents=True, exist_ok=True)
    
    # Avaliar e filtrar
    quality_evaluator = QualityMetrics()
    
    image_files = list((dataset_dir / "images").glob("*.jpg"))
    kept_images = []
    rejected_images = []
    
    for img_path in tqdm(image_files, desc="Filtrando por qualidade"):
        try:
            # Carregar e avaliar
            image = Image.open(img_path)
            report = quality_evaluator.evaluate_image_quality(image)
            
            if report.overall_score >= quality_threshold:
                # Copiar imagem
                dest_img = output_dir / "images" / img_path.name
                image.save(dest_img)
                
                # Copiar metadata se existir
                metadata_src = dataset_dir / "metadata" / (img_path.stem + ".json")
                metadata_dest = output_dir / "metadata" / (img_path.stem + ".json")
                
                if metadata_src.exists():
                    import shutil
                    shutil.copy2(metadata_src, metadata_dest)
                
                kept_images.append({
                    'filename': img_path.name,
                    'score': report.overall_score
                })
            else:
                rejected_images.append({
                    'filename': img_path.name,
                    'score': report.overall_score
                })
                
        except Exception as e:
            print(f"Erro processando {img_path.name}: {e}")
            continue
    
    print(f"\n✅ Filtragem concluída!")
    print(f"   Mantidas: {len(kept_images)}/{len(image_files)} ({len(kept_images)/len(image_files)*100:.1f}%)")
    print(f"   Rejeitadas: {len(rejected_images)} ({len(rejected_images)/len(image_files)*100:.1f}%)")
    
    if kept_images:
        avg_score = np.mean([img['score'] for img in kept_images])
        print(f"   Score médio das mantidas: {avg_score:.3f}")
    
    # Salvar relatório
    filter_report = {
        'filter_threshold': quality_threshold,
        'original_count': len(image_files),
        'kept_count': len(kept_images),
        'rejected_count': len(rejected_images),
        'kept_images': kept_images,
        'rejected_images': rejected_images
    }
    
    with open(output_dir / "filter_report.json", 'w') as f:
        json.dump(filter_report, f, indent=2)
    
    return output_path

# Interface de filtragem
filter_threshold_widget = widgets.FloatSlider(
    value=0.7,
    min=0.4,
    max=0.9,
    step=0.05,
    description='Threshold:'
)

filter_output_widget = widgets.Text(
    value='/content/filtered_dataset',
    description='Output:'
)

filter_btn = widgets.Button(
    description='🎯 Filtrar Dataset',
    button_style='warning'
)

filter_output = widgets.Output()

def filter_callback(btn):
    with filter_output:
        clear_output()
        filter_dataset_by_quality(
            dataset_path_widget.value,
            filter_threshold_widget.value,
            filter_output_widget.value
        )

filter_btn.on_click(filter_callback)

display(widgets.VBox([
    widgets.HTML("<h3>🎯 Filtragem por Qualidade</h3>"),
    filter_threshold_widget,
    filter_output_widget,
    filter_btn,
    filter_output
]))

## 📊 Comparação com Dataset Real

In [ ]:
def compare_synthetic_vs_real(synthetic_path, real_path=None, sample_size=30):
    """Compara qualidade entre datasets sintético e real"""
    
    if not real_path:
        print("⚠️ Dataset real não fornecido - comparação não disponível")
        return
    
    print(f"📊 Comparando datasets...")
    print(f"Sintético: {synthetic_path}")
    print(f"Real: {real_path}")
    
    quality_evaluator = QualityMetrics()
    
    # Analisar dataset sintético
    synthetic_scores = []
    synthetic_images = list(Path(synthetic_path).glob("images/*.jpg"))[:sample_size]
    
    for img_path in tqdm(synthetic_images, desc="Analisando sintético"):
        try:
            image = Image.open(img_path)
            report = quality_evaluator.evaluate_image_quality(image)
            synthetic_scores.append(report.overall_score)
        except:
            continue
    
    # Analisar dataset real
    real_scores = []
    real_images = list(Path(real_path).glob("**/*.jpg"))[:sample_size]
    
    if real_images:
        for img_path in tqdm(real_images, desc="Analisando real"):
            try:
                image = Image.open(img_path)
                report = quality_evaluator.evaluate_image_quality(image)
                real_scores.append(report.overall_score)
            except:
                continue
    
    # Comparação
    if synthetic_scores and real_scores:
        print(f"\n📊 COMPARAÇÃO DE QUALIDADE")
        print("=" * 40)
        print(f"Sintético: {np.mean(synthetic_scores):.3f} ± {np.std(synthetic_scores):.3f}")
        print(f"Real: {np.mean(real_scores):.3f} ± {np.std(real_scores):.3f}")
        
        # Teste estatístico
        from scipy import stats
        t_stat, p_value = stats.ttest_ind(synthetic_scores, real_scores)
        print(f"\nTeste t: p-value = {p_value:.4f}")
        
        if p_value < 0.05:
            print("Diferença estatisticamente significativa")
        else:
            print("Sem diferença estatística significativa")
        
        # Visualização
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.hist(synthetic_scores, alpha=0.7, label='Sintético', bins=15)
        plt.hist(real_scores, alpha=0.7, label='Real', bins=15)
        plt.xlabel('Score de Qualidade')
        plt.ylabel('Frequência')
        plt.title('Distribuição de Qualidade')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.boxplot([synthetic_scores, real_scores], labels=['Sintético', 'Real'])
        plt.ylabel('Score de Qualidade')
        plt.title('Comparação Boxplot')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    else:
        print("❌ Não foi possível realizar comparação")

# Interface de comparação
real_path_widget = widgets.Text(
    value='',
    placeholder='/path/to/real/dataset',
    description='Dataset Real:'
)

compare_btn = widgets.Button(
    description='📊 Comparar',
    button_style='info'
)

compare_output = widgets.Output()

def compare_callback(btn):
    with compare_output:
        clear_output()
        compare_synthetic_vs_real(
            dataset_path_widget.value,
            real_path_widget.value if real_path_widget.value else None
        )

compare_btn.on_click(compare_callback)

display(widgets.VBox([
    widgets.HTML("<h3>📊 Comparação Sintético vs Real</h3>"),
    real_path_widget,
    compare_btn,
    compare_output
]))